In [1]:
# !nvcc --version # cuda version

In [2]:
# # For CUDA 12.x (latest)
# !pip install faiss-gpu

# # For CUDA 11.x
# !pip install faiss-gpu-cuda11

# # For CUDA 10.x
# !pip install faiss-gpu-cuda10

In [ ]:
!pip install faiss-cpu # Cpu version

In [4]:
import numpy as np
import faiss

# Let's create a matrix of vectors (like embeddings)
d = 5    # dimension of vectors
nb = 10  # number of vectors in the database

np.random.seed(42)
data = np.random.random((nb, d)).astype('float32')

# Create an index using L2 (Euclidean) distance
index = faiss.IndexFlatL2(d)

# Add our vectors to the index
index.add(data)

# Let's create a query vector
query = np.random.random((1, d)).astype('float32')

# Search for the 3 most similar vectors
k = 3
distances, indices = index.search(query, k)

print("Query vector:\n", query)
print("Indices of nearest neighbors:", indices)
print("Distances:", distances)

Query vector:
 [[0.96958464 0.77513283 0.93949896 0.89482737 0.5979    ]]
Indices of nearest neighbors: [[0 9 5]]
Distances: [[0.71094024 0.7767615  0.9415597 ]]


In [9]:
d = 130
nb = 5000
np.random.seed(42)

data = np.random.random((nb, d)).astype('float32')

# Graph-based ANN
index = faiss.IndexHNSWFlat(d, 32)  # 32 = number of neighbors per node

index.add(data)

query = np.random.random((1, d)).astype('float32')
k = 4

distances, indices = index.search(query, k)

print("HNSW - Indices:", indices)
print("HNSW - Distances:", distances)

HNSW - Indices: [[4426 2989 2411 2424]]
HNSW - Distances: [[13.049747 13.362212 13.374662 13.538497]]


In [28]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle

# Step 1: Load pretrained sentence embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # 384-dim vectors

# Step 2: Example texts
texts = [
    "I love machine learning",
    "Faiss is great for similarity search",
    "Python is my favorite language",
    "Transformers are powerful for NLP",
    "Deep learning is a game changer"
]

# Step 3: Convert texts to vectors
embeddings = model.encode(texts, convert_to_numpy=True).astype('float32')  # shape: (n, d)

print("Vector shape:", embeddings.shape)  # (5, 384)

# Step 4: Create index
dimension = embeddings.shape[1]

# # ----Cosine similarity------------------------------------------------------------
# # If we are going to use IndexFlatIP (Cosine similarity) we have to Normalize embeddings
# # before adding to IndexFlatIP For better search quality
# faiss.normalize_L2(embeddings)
# index = faiss.IndexFlatIP(dimension)
# # ---------------------------------------------------------------------------------

# ----Graph-based ANN----------------------------------------------------------------
# index = faiss.IndexHNSWFlat(dimension, 32)  # You can replace with any other index (e.g. IVFFlat)
# -----------------------------------------------------------------------------------

# ----Euclidean distance-------------------------------------------------------------
index = faiss.IndexFlatL2(dimension)  # You can replace with any other index (e.g. IVFFlat)
# -----------------------------------------------------------------------------------

# Optional: Map texts to IDs (if needed)
text_id_map = {i: texts[i] for i in range(len(texts))}

# Step 5: Add embeddings to index
index.add(embeddings)

# Step 6: Save index and ID map
faiss.write_index(index, "text_index.faiss")

# Save the text ID map using pickle
with open("id_map.pkl", "wb") as f:
    pickle.dump(text_id_map, f)

print("✅ Index saved with", len(texts), "texts.")

Vector shape: (5, 384)
✅ Index saved with 5 texts.


In [34]:
# Step 1: Load index and ID map
index = faiss.read_index("text_index.faiss")

with open("id_map.pkl", "rb") as f:
    text_id_map = pickle.load(f)

# Step 2: Convert new query to vector
query = ["What is deep learning?", "what is machine learning"]
query_vec = model.encode(query, convert_to_numpy=True).astype('float32')

# Step 3: Search
k = 3
D, I = index.search(query_vec, k)

print("Query:", query)
print("Top matches:")
for i, idx in enumerate(I[0]):
    print(f"{i+1}. {text_id_map[idx]} (distance={D[0][i]:.4f})")

print("-"*100)

for i, idx in enumerate(I[1]):
    print(f"{i+1}. {text_id_map[idx]} (distance={D[0][i]:.4f})")

Query: ['What is deep learning?', 'what is machine learning']
Top matches:
1. Deep learning is a game changer (distance=0.6720)
2. I love machine learning (distance=1.1027)
3. Transformers are powerful for NLP (distance=1.5337)
----------------------------------------------------------------------------------------------------
1. I love machine learning (distance=0.6720)
2. Deep learning is a game changer (distance=1.1027)
3. Transformers are powerful for NLP (distance=1.5337)
